In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

contradictory_my_dear_watson_path = kagglehub.competition_download('contradictory-my-dear-watson')

print('Data source import complete.')

100%|██████████| 1.78M/1.78M [00:01<00:00, 1.66MB/s]

Extracting files...
Data source import complete.


Goal: To finetune a roBERTa classification model in order to identify the thing!

First, have to get into huggingface and nab a model.

In [ ]:
from huggingface_hub import login

login()

Ok with the data obtained, now lets actually get our hands dirty. First step, get the nec. libraries

In [ ]:
!pip install transformers datasets huggingface_hub tensorboard wandb evaluate
!pip install -U transformers
!pip install --upgrade accelerate
!pip install huggingface_hub optuna
!sudo apt-get install git-lfs --yes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" an

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    XLMRobertaTokenizerFast,
    XLMRobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
torch.cuda.is_available()

True

In [ ]:
dataset = load_dataset(contradictory_my_dear_watson_path)

tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [ ]:
from datasets import ClassLabel

# def tokenize function
def tokenize_function(example):
    return tokenizer(example['premise'],
                     example['hypothesis'],
                     truncation=True,
                     padding='max_length')

# tokenize all splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# for each split, cast & rename
for split in tokenized_datasets:
    tokenized_datasets[split] = (
        tokenized_datasets[split]
        # convert float→int data + update schema
        .cast_column("label", ClassLabel(num_classes=3))
        # rename to the Trainer’s expected key
        .rename_column("label", "labels")
    )

# after cast/rename on train:
split = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=42)
tokenized_datasets["train"] = split["train"]
tokenized_datasets["validation"] = split["test"]

# Now bind Torch format
tokenized_datasets.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/12120 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/12120 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5195 [00:00<?, ? examples/s]

In [ ]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

# Define compute_metrics func
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=preds, references=labels)


In [ ]:
import optuna

model_checkpoint = "xlm-roberta-base"

def model_init():
    return XLMRobertaForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=3
    )

# Shared TrainingArguments
training_args = TrainingArguments(
    output_dir=f"./results/{model_checkpoint.replace('/', '_')}",
    eval_strategy    = "epoch",
    save_strategy          = "epoch",
    load_best_model_at_end = True,
    metric_for_best_model  = "eval_accuracy",
    greater_is_better      = True,
    save_total_limit       = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 16,
    num_train_epochs       = 3,
    logging_strategy       = "epoch",
)

# define the hyperparameter
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 2e-5]),
        "weight_decay":  trial.suggest_categorical("weight_decay",  [0.0, 0.01]),
    }

# init the trainer
trainer = Trainer(
    model_init      = model_init,
    args            = training_args,
    train_dataset   = tokenized_datasets['train'],
    eval_dataset    = tokenized_datasets['validation'],
    compute_metrics = compute_metrics,
    callbacks       = [EarlyStoppingCallback(early_stopping_patience=1)],
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


74c9f5648fb38bc53a50b6fffe72f99dac81d881 <- api key

In [ ]:
# Run hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    n_trials=4,
    compute_objective=lambda metrics: metrics["eval_accuracy"],
)

print("✔︎ Best hyperparameters:", best_run.hyperparameters)

# update Trainer args with the best params found in the search
for name, value in best_run.hyperparameters.items():
    setattr(trainer.args, name, value)

trainer.train()

[I 2025-05-07 00:50:04,007] A new study created in memory with name: no-name-03799277-f98d-4d60-b220-230fa6d383fe
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bwoodsii (bwoodsii-stevens-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

[I 2025-05-07 01:38:01,661] Trial 3 finished with value: 0.7458745874587459 and parameters: {'learning_rate': 1e-05, 'weight_decay': 0.01}. Best is trial 2 with value: 0.754950495049505.


✔︎ Best hyperparameters: {'learning_rate': 2e-05, 'weight_decay': 0.01}


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.027000,0.858362,0.634488
2,0.769800,0.649730,0.736799
3,0.599800,0.632448,0.745050


TrainOutput(global_step=2046, training_loss=0.7988520050794614, metrics={'train_runtime': 705.0634, 'train_samples_per_second': 46.413, 'train_steps_per_second': 2.902, 'total_flos': 8610123481731072.0, 'train_loss': 0.7988520050794614, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6324478387832642,
 'eval_accuracy': 0.745049504950495,
 'eval_runtime': 7.3753,
 'eval_samples_per_second': 164.333,
 'eval_steps_per_second': 10.305,
 'epoch': 3.0}

Now its just a matter of saving the model!

In [ ]:
trainer.push_to_hub("Woodsii/BERT_base_kaggle_watson")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Upload 28 LFS files:   0%|          | 0/28 [00:00<?, ?it/s]

optimizer.pt:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

...

events.out.tfevents.1746581172.d48379239e32.6058.3:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

events.out.tfevents.1746581882.d48379239e32.6058.4:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

events.out.tfevents.1746582595.d48379239e32.6058.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Woodsii/xlm-roberta-base/commit/6b1048dec9e0080a0fc0dd24972d68119f49b10b', commit_message='Woodsii/BERT_base_kaggle_watson', commit_description='', oid='6b1048dec9e0080a0fc0dd24972d68119f49b10b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Woodsii/xlm-roberta-base', endpoint='https://huggingface.co', repo_type='model', repo_id='Woodsii/xlm-roberta-base'), pr_revision=None, pr_num=None)